In [83]:
#Problem: With the insurgence of indie games on Steam, it is hard to determine what would make a successful indie game. 
#Questions: What genres of indie games are most popular? Which ones are oversaturated? If I wanted to make a successful indie game, what genre should I make it in?
#Data Sources: SteamSpy, Storefront API, Steam Web API
#Steam Web API (key: DCD8AAAB9BA14A860C20C9138E54CB50, domain name: blackmoose)

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sklearn

In [85]:
#importing data from csv files
steamspy_df = pd.read_csv('steamspy_all_games.csv')
storefront_df_raw = pd.read_csv('storefront_data.csv')

In [86]:
print(storefront_df_raw.head(2))
print(storefront_df_raw['genres'].head(2).tolist())
#this is showing that the json is stringified, so we need to convert it to a real list before we can use it
storefront_df_raw['genres_parsed'] = storefront_df_raw['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
storefront_df_raw['genres_parsed'].head(2).tolist()

     appid                                             genres  type  \
0      730  [{'id': '1', 'description': 'Action'}, {'id': ...  game   
1  1172470  [{'id': '1', 'description': 'Action'}, {'id': ...  game   

                                     release_date  
0  {'coming_soon': False, 'date': '21 Aug, 2012'}  
1   {'coming_soon': False, 'date': 'Nov 4, 2020'}  
["[{'id': '1', 'description': 'Action'}, {'id': '37', 'description': 'Free To Play'}]", "[{'id': '1', 'description': 'Action'}, {'id': '25', 'description': 'Adventure'}, {'id': '37', 'description': 'Free To Play'}]"]


[[{'id': '1', 'description': 'Action'},
  {'id': '37', 'description': 'Free To Play'}],
 [{'id': '1', 'description': 'Action'},
  {'id': '25', 'description': 'Adventure'},
  {'id': '37', 'description': 'Free To Play'}]]

In [87]:
storefront_df = storefront_df_raw.drop(columns=['genres', 'release_date', 'genres_parsed'])
print(storefront_df_raw['genres'].head())
storefront_df['genres_list'] = storefront_df_raw['genres_parsed'].apply(lambda x: [g['description'] for g in x if 'description' in g] if isinstance(x, list) else [])
print(storefront_df.columns)
print(storefront_df.head())
print(len(storefront_df))

0    [{'id': '1', 'description': 'Action'}, {'id': ...
1    [{'id': '1', 'description': 'Action'}, {'id': ...
2    [{'id': '1', 'description': 'Action'}, {'id': ...
3    [{'id': '1', 'description': 'Action'}, {'id': ...
4    [{'id': '1', 'description': 'Action'}, {'id': ...
Name: genres, dtype: object
Index(['appid', 'type', 'genres_list'], dtype='object')
     appid  type                                        genres_list
0      730  game                             [Action, Free To Play]
1  1172470  game                  [Action, Adventure, Free To Play]
2   578080  game  [Action, Adventure, Massively Multiplayer, Fre...
3  1623730  game      [Action, Adventure, Indie, RPG, Early Access]
4      440  game                             [Action, Free To Play]
86390


In [88]:
print(steamspy_df.describe())
print(steamspy_df.columns)
print(steamspy_df.head())
print(len(steamspy_df))

              appid  score_rank      positive      negative     userscore  \
count  8.652800e+04   43.000000  8.652800e+04  8.652800e+04  86528.000000   
mean   1.530538e+06   99.232558  1.470238e+03  2.419816e+02      0.038508   
std    9.059966e+05    0.648706  3.264946e+04  6.194527e+03      1.766213   
min    1.000000e+01   98.000000  0.000000e+00  0.000000e+00      0.000000   
25%    7.795775e+05   99.000000  5.000000e+00  1.000000e+00      0.000000   
50%    1.382915e+06   99.000000  1.800000e+01  5.000000e+00      0.000000   
75%    2.212212e+06  100.000000  9.500000e+01  2.700000e+01      0.000000   
max    3.744990e+06  100.000000  7.642084e+06  1.173003e+06    100.000000   

       average_forever  average_2weeks  median_forever  median_2weeks  \
count     86528.000000    86528.000000    86528.000000   86528.000000   
mean        223.022605       14.795373      191.611166      15.662479   
std        4223.636880      213.135683     5336.906312     241.607244   
min           

In [89]:
#Because not all rows were successfully extracted from API, there is a mismatch in the number of rows between the two dataframes. For this reason, we will use merge on dataframes to combine them.
df = pd.merge(storefront_df, steamspy_df, left_on='appid', right_on='appid', how='inner')
print(df.head())
print(len(df))
#RED FLAG: 
#there are 93787 which is larger than the two individual dataframes, so it is likely that some rows were duplicated in the merge, or there are some mismatched appids in the two dataframes.
#Should not be more than the individual dataframes, due to the inner join.

     appid  type                                        genres_list  \
0      730  game                             [Action, Free To Play]   
1  1172470  game                  [Action, Adventure, Free To Play]   
2   578080  game  [Action, Adventure, Massively Multiplayer, Fre...   
3  1623730  game      [Action, Adventure, Indie, RPG, Early Access]   
4      440  game                             [Action, Free To Play]   

                               name         developer        publisher  \
0  Counter-Strike: Global Offensive             Valve            Valve   
1                      Apex Legends           Respawn  Electronic Arts   
2               PUBG: BATTLEGROUNDS  PUBG Corporation    KRAFTON, Inc.   
3                          Palworld        Pocketpair       Pocketpair   
4                   Team Fortress 2             Valve            Valve   

   score_rank  positive  negative  userscore                      owners  \
0         NaN   7642084   1173003          0  100,00

In [90]:
#Looking why there is more in the merged dataframe than in the individual dataframes
#checking for duplicates in the individual dataframes
print("Storefront duplicates previous to remove:", storefront_df['appid'].duplicated().sum(), "Length pre-duplicates:", len(storefront_df))
print("Steamspy duplicates previous to remove:", steamspy_df['appid'].duplicated().sum(), "Length pre-duplicates:", len(steamspy_df))
storefront_df = storefront_df.drop_duplicates(subset='appid')
steamspy_df = steamspy_df.drop_duplicates(subset='appid')
print("Storefront duplicates after drop:", storefront_df['appid'].duplicated().sum(), "Length post-duplicates:", len(storefront_df))
print("Steamspy duplicates after drop:", steamspy_df['appid'].duplicated().sum(), "Length post-duplicates:", len(steamspy_df))

df = pd.merge(storefront_df, steamspy_df, left_on='appid', right_on='appid', how='inner')
print("Merged duplicates after drop:", df['appid'].duplicated().sum(), "Length post-duplicates:", len(df))
if len(df) > len(storefront_df) and len(df) > len(steamspy_df):
    print("There are still more rows in the merged dataframe than in the individual dataframes. There may be some mismatched appids in the two dataframes.")
else:
    print("The merged dataframe has the same number or fewer rows as the individual dataframes. There are no mismatched appids.")

Storefront duplicates previous to remove: 3760 Length pre-duplicates: 86390
Steamspy duplicates previous to remove: 3773 Length pre-duplicates: 86528
Storefront duplicates after drop: 0 Length post-duplicates: 82630
Steamspy duplicates after drop: 0 Length post-duplicates: 82755
Merged duplicates after drop: 0 Length post-duplicates: 82547
The merged dataframe has the same number or fewer rows as the individual dataframes. There are no mismatched appids.


In [91]:
#Looking for NAs in the merged dataframe
print("NAs in merged dataframe:", df.isna().sum())
print("percent of NAs in merged dataframe:", df.isna().sum() / len(df) * 100)

#Missing Data handling
#score_rank has a lot of NAs, so we will drop the column
#Name, Developer, and Publisher have some NAs, so we will drop those rows since we cannot impute them meaningfully
df = df.drop(columns=['score_rank'])
df = df.dropna()
print("NAs in merged dataframe after drop:", df.isna().sum())
print("Length of merged dataframe after drop:", len(df))

NAs in merged dataframe: appid                  0
type                   0
genres_list            0
name                  11
developer            209
publisher            483
score_rank         82510
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                  0
initialprice           0
discount               0
ccu                    0
dtype: int64
percent of NAs in merged dataframe: appid               0.000000
type                0.000000
genres_list         0.000000
name                0.013326
developer           0.253189
publisher           0.585121
score_rank         99.955177
positive            0.000000
negative            0.000000
userscore           0.000000
owners              0.000000
average_forever     0.000000
average_2weeks      0.000000
median_forever      0.000000
median_2weeks       0.000000
price           

In [ ]:
#looking at what columns we need to clean next
print(df.dtypes)
print(df.head(5))
#Owners is a string, with a range of values (e.g. "100,000 to 200,000"). How necessary is this column? If we keep it, we will need to convert it to a numerical value. But there is no way
#to parse the information in a meaningful way, since there will be a many repeated values for different games. Will this add noise or meaninful information to the dataset?
#Genres_list needs to be encoded to a numerical value, since it is a list of strings. We can use  encoding to do this.
#Additionally, the "indie" column from the  encoding + the type will be used to filter the dataframe to only include indie games.
#CCU variable also poses a problem with analysis, so we will approach it with caution. CCU = Concurrent Users, which is a snapshot of the number of users playing the game at the time of import. 
#This can be useful or can add unnecessary noise because it is heavily affected by multiple factors, such as time of day, day of the week, season, whether it is on sale, popularity by region, etc.
#We will keep it for now, and see how much information it adds to the models.

#NEXT STEPS:
#1. Before looking further at the data, we need to  encode the genres_list column to create a new dataframe with the genres as columns.
#2. We will then filter the dataframe to only include indie games, and create a new dataframe with only the indie games.
#3. Futher EDA on the subset of indie games to look at distributions of other columns. 
#4. Re-assess what next steps are needed after EDA.


appid                int64
type                object
genres_list         object
name                object
developer           object
publisher           object
positive             int64
negative             int64
userscore            int64
owners              object
average_forever      int64
average_2weeks       int64
median_forever       int64
median_2weeks        int64
price              float64
initialprice       float64
discount           float64
ccu                  int64
dtype: object
     appid  type                                        genres_list  \
0      730  game                             [Action, Free To Play]   
1  1172470  game                  [Action, Adventure, Free To Play]   
2   578080  game  [Action, Adventure, Massively Multiplayer, Fre...   
3  1623730  game      [Action, Adventure, Indie, RPG, Early Access]   
4      440  game                             [Action, Free To Play]   

                               name         developer        publisher  \

In [ ]:
#separating out the genres_list column to view what breakdown is
genres_df = df['genres_list'].str.join('|').str.get_dummies()
print(genres_df.columns)
print(len(genres_df.columns))

#Unexpected issue of multiple languages for the genres, so two ways to approach this:
#1. Easier method: Use the ID of the genres from the Steam API, which might be consistent across languages.
#1a. This would require us to look up the IDs for each genre, and then create a mapping from the genre name to the ID.
#2. More complex method: Use the genres as they are, but this will require us to clean the data further to ensure that the genres are consistent across languages.
#2a. This would require us to create a mapping from the genre name to a standard name, and then  encode the standard names.

#we will obviously start with the easier method.

Index(['Abenteuer', 'Acceso anticipado', 'Acción', 'Accounting',
       'Acesso Antecipado', 'Action', 'Adventure', 'Akció', 'Akcja',
       'Animation & Modeling', 'Audio Production', 'Aventura', 'Avventura',
       'Ação', 'Casual', 'Corridas', 'Corse', 'Design & Illustration',
       'Early Access', 'Education', 'Estratégia', 'Free To Play', 'GDR',
       'Game Development', 'Gelegenheitsspiele', 'Gore', 'Indie',
       'Indépendant', 'Massively Multiplayer', 'Movie',
       'Multigiocatore di massa', 'Multijugador masivo', 'Niezależne',
       'Nudity', 'Passatempo', 'Photo Editing', 'Przygodowe', 'RPG', 'Racing',
       'Rekreacyjne', 'Rennspiele', 'Rol', 'Rollenspiel', 'Sexual Content',
       'Short', 'Simulation', 'Simulationen', 'Simulazione', 'Simulação',
       'Software Training', 'Sport', 'Sports', 'Strategy', 'Stratégia',
       'Symulacje', 'Szimuláció', 'Utilities', 'Video Production', 'Violent',
       'Web Publishing', 'Αγώνες ταχύτητας', 'Δράση', 'Προσομοίωση', 'Инди

In [94]:
#Now instead of parsing genres from the descriptions, we will parse the genres from the id.
#If there is the same number of columns created as the number of columns above, then we will need to try the other method (since ids would not be consistent across languages).
#copy our line from above and change 'description' to 'id'

storefront_df['genres_list_id'] = storefront_df_raw['genres_parsed'].apply(lambda x: [g['id'] for g in x if 'id' in g] if isinstance(x, list) else [])
genres_df_id = storefront_df['genres_list_id'].str.join('|').str.get_dummies()
print(genres_df_id.columns)
print(len(genres_df_id.columns))

#This looks consistent across languages, sicne there are 60 less columns created
#Next step is duplicate this method into the main dataframe
#Create a mapping dictionary to then add description in EN to each id.


Index(['1', '18', '2', '23', '25', '28', '29', '3', '37', '4', '50', '51',
       '52', '53', '54', '55', '56', '57', '58', '59', '60', '70', '71', '72',
       '73', '74', '80', '83', '9'],
      dtype='object')
29


In [95]:
#merging again and dropping pertinent columns to replace the genre_list with the genre_list_id in the merged df
df = pd.merge(storefront_df, steamspy_df, left_on='appid', right_on='appid', how='inner')
print("Merged duplicates after drop:", df['appid'].duplicated().sum(), "Length post-duplicates:", len(df))
if len(df) > len(storefront_df) and len(df) > len(steamspy_df):
    print("There are still more rows in the merged dataframe than in the individual dataframes. There may be some mismatched appids in the two dataframes.")
else:
    print("The merged dataframe has the same number or fewer rows as the individual dataframes. There are no mismatched appids.")
df = df.drop(columns='genres_list')
print(df.columns)
print(len(df))
#double checking that the rows match to the merge before to ensure no issues with this process
if len(df) == 82547:
    print("No Flags")
else:
    print("ERROR")

Merged duplicates after drop: 0 Length post-duplicates: 82547
The merged dataframe has the same number or fewer rows as the individual dataframes. There are no mismatched appids.
Index(['appid', 'type', 'genres_list_id', 'name', 'developer', 'publisher',
       'score_rank', 'positive', 'negative', 'userscore', 'owners',
       'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks',
       'price', 'initialprice', 'discount', 'ccu'],
      dtype='object')
82547
No Flags


In [ ]:
#Creating a dictionary to map the ids to the EN descriptions so results are interpretable
#ASSUMPTION: English descriptions will be in the front. This method takes the first description for each id to map them together, if there is an error will have to manually change.
all_genres = storefront_df_raw['genres_parsed'].dropna().explode()
genre_dicts = [g for g in all_genres if isinstance(g, dict) and 'id' in g and 'description' in g]
genre_map = {}
for g in genre_dicts:
    genre_map[g['id']] = genre_map.get(g['id'], g['description'])
print(genre_map.values())
if len(genre_map.keys()) == len(genres_df_id.columns):
    print("No Flags")
else:
    print("ERROR")
#Looks like that maps correctly to the number of ids that were made in the  encoding above. Visual check shows that all the of the descriptions were in english

dict_values(['Action', 'Free To Play', 'Adventure', 'Massively Multiplayer', 'Indie', 'RPG', 'Early Access', 'Casual', 'Simulation', 'Animation & Modeling', 'Design & Illustration', 'Photo Editing', 'Utilities', 'Strategy', 'Sports', 'Racing', 'Video Production', 'Education', 'Game Development', 'Audio Production', 'Violent', 'Software Training', 'Web Publishing', 'Nudity', 'Gore', 'Movie', 'Sexual Content', 'Accounting', 'Short'])
No Flags


In [ ]:
#Mapping the english description to the genres_list_id column, so that we create a column with english descriptions for  encoding.
df['genres_list'] = df['genres_list_id'].apply(
    lambda id_list: [genre_map.get(i, f"Unknown-{i}") for i in id_list]
)

#Double checking to ensure accuracy
#encoding the genres_list column
genres_df = df['genres_list'].str.join('|').str.get_dummies()
print(genres_df.columns)
print(len(genres_df.columns))
print(genres_df.head(5))
print(df['genres_list'].head())
#printed both to triple check that mapping was done correctly. Visual check shows correctness

Index(['Accounting', 'Action', 'Adventure', 'Animation & Modeling',
       'Audio Production', 'Casual', 'Design & Illustration', 'Early Access',
       'Education', 'Free To Play', 'Game Development', 'Gore', 'Indie',
       'Massively Multiplayer', 'Movie', 'Nudity', 'Photo Editing', 'RPG',
       'Racing', 'Sexual Content', 'Short', 'Simulation', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production', 'Violent',
       'Web Publishing'],
      dtype='object')
29
   Accounting  Action  Adventure  Animation & Modeling  Audio Production  \
0           0       1          0                     0                 0   
1           0       1          1                     0                 0   
2           0       1          1                     0                 0   
3           0       1          1                     0                 0   
4           0       1          0                     0                 0   

   Casual  Design & Illustration  Early Access

In [ ]:
#Cleaning up again before encoding
print(df.columns)
df = df.drop(columns=['genres_list_id', 'score_rank'])
df = df.dropna()
print(df.isna().sum())

Index(['appid', 'type', 'name', 'developer', 'publisher', 'positive',
       'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'genres_list'],
      dtype='object')


KeyError: "['genres_list_id', 'score_rank'] not found in axis"

In [105]:
print(df['genres_list'])

0                                   [Action, Free To Play]
1                        [Action, Adventure, Free To Play]
2        [Action, Adventure, Massively Multiplayer, Fre...
3            [Action, Adventure, Indie, RPG, Early Access]
4                                   [Action, Free To Play]
                               ...                        
82542                                   [Adventure, Indie]
82543                                          [Adventure]
82544                                 [Casual, Simulation]
82545                                      [Casual, Indie]
82546                                      [Casual, Indie]
Name: genres_list, Length: 81924, dtype: object


In [108]:
#encoding
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['genres_list'])

genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=df.index)
df_encoded = pd.concat([df, genres_df], axis=1)
print(df_encoded.columns)

Index(['appid', 'type', 'name', 'developer', 'publisher', 'positive',
       'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'genres_list', 'Accounting', 'Action', 'Adventure',
       'Animation & Modeling', 'Audio Production', 'Casual',
       'Design & Illustration', 'Early Access', 'Education', 'Free To Play',
       'Game Development', 'Gore', 'Indie', 'Massively Multiplayer', 'Movie',
       'Nudity', 'Photo Editing', 'RPG', 'Racing', 'Sexual Content', 'Short',
       'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Violent', 'Web Publishing'],
      dtype='object')


In [ ]:
#filtering to just Indie games to do some further EDA
df_indie = df_encoded[(df_encoded['Indie'] == 1) & (df_encoded['type'] == 'game')]
print(len(df_indie))

58158
